<a href="https://colab.research.google.com/github/9-coding/ComputerVision_TeamProject/blob/main/ActiveLerning1_refer_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [5]:
import os
# 각자 환경에서 이 부분만 변경하여 사용하면 됩니다.
os.chdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50')
print(os.getcwd())


/content/gdrive/MyDrive/CUB_200_2011_repackage_class50


In [6]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=7cc5cdcb500ad2cf05a305f664bbeccf49a138f502e0ffb8dab897976261b362
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [7]:
!wandb login f9fc6c65de4904021a59f4c8c60b90961f6ec1fb

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [15]:
import os
import time
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import random
import numpy as np

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(42)

import wandb
wandb.init(project='activelearning1_doyeon')

### GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

### Custom Dataset ###
class CUB2011(Dataset):
  def __init__(self, transform, mode='train'):
    self.transform = transform
    self.mode = mode

    if self.mode == 'train':
      self.image_folder = os.listdir('./datasets/train')
    elif self.mode == 'valid':
      self.image_folder = os.listdir('./datasets/valid')
    elif self.mode == 'test':
      self.image_folder = os.listdir('./datasets/test')

  def __len__(self):
    return len(self.image_folder)

  def __getitem__(self, idx):
    img_path= self.image_folder[idx]
    img = Image.open(os.path.join('./datasets', self.mode, img_path)).convert('RGB')
    img = self.transform(img)
    label = img_path.split('_')[-1].split('.')[0]
    label = int(label)
    return (img, label)

### Data Preprocessing ###
transforms_train = transforms.Compose([transforms.Resize((448, 448)), transforms.ToTensor()])
transforms_valtest = transforms.Compose([transforms.Resize((448, 448)), transforms.ToTensor()])

BATCH_SIZE = 32
train_set = CUB2011(mode='train', transform=transforms_train)
val_set = CUB2011(mode= 'valid', transform=transforms_valtest)
test_set = CUB2011(mode='test', transform=transforms_valtest)
print('Num of each dataset:', len(train_set), len(val_set), len(test_set))

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader (val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader (test_set, batch_size=BATCH_SIZE, shuffle=False)

print("Loaded dataloader")

### Model / Optimzier ###
EPOCH = 30
lr = 0.1
model = models.resnet18(pretrained=True)

### Transfer Learning ###
num_features = model.fc.in_features
model.fc = nn. Linear(num_features, 50)
model.to(DEVICE)
optimizer = optim.SGD (model.parameters(), lr=lr)
print("Created a learning model and optimizer")


### Train/Evaluation ###
def train(model, train_loader, optimizer, epoch):
  model.train()
  for i, (image, target) in enumerate(train_loader):
    image, target = image.to(DEVICE), target.to(DEVICE)
    output = model(image)
    optimizer.zero_grad()
    train_loss = F.cross_entropy(output, target).to(DEVICE)

    train_loss.backward()
    optimizer.step()

    if i % 10 == 0:
      print(f'Train Epoch {epoch} [{i}/{len(train_loader)}]\tLoss: {train_loss.item():.6f}')
  return train_loss

def evaluate(model, val_loader):
  model.eval()
  eval_loss = 0
  correct = 0
  with torch.no_grad():
    for i, (image, target) in enumerate (val_loader):
      image, target = image.to(DEVICE), target.to(DEVICE)
      output = model(image)
      eval_loss += F.cross_entropy (output, target, reduction='sum').item()
      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as (pred)).sum().item()
    eval_loss /= len(val_loader.dataset)
    eval_accuracy = 100 * correct / len(val_loader.dataset)
    return eval_loss, eval_accuracy

### Main ###
start = time.time()
best = 0
for epoch in range(EPOCH):
  train_loss = train(model, train_loader, optimizer, epoch)
  val_loss, val_accuracy = evaluate(model, val_loader)

  # Logging for model with augmentation
  wandb.log({
        'lr': lr,
        'BATCH_SIZE': BATCH_SIZE,
        'train_loss': train_loss.item(),
        'val_loss': val_loss,
        'val_accuracy': val_accuracy,
    })

  # Save best model
  if val_accuracy > best:
    best = val_accuracy
    torch.save(model.state_dict(), "./best_model.pth")
  print(f'[{epoch}] Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}%')

# Test result
test_loss, test_accuracy = evaluate(model, test_loader)
wandb.log({'test_accuracy': test_accuracy})
print(f'[FINAL] Test Loss {test_loss:.4f}, Accuracy: {test_accuracy:.4f}%')
end = time.time()
elasped_time = end - start

print("Best Accuracy: ", best)
print(f"Elasped Time: {int(elasped_time/3600)}h, {int(elasped_time/60)}m, {int(elasped_time%60)}s")
print(f"time: {int (elasped_time/3600)}h, {int(elasped_time/60)}m, {int(elasped_time%60)}s")

Random seed set as 42


cuda
Num of each dataset: 2360 296 298
Loaded dataloader


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Created a learning model and optimizer
Train Epoch 0 [0/74]	Loss: 4.035903
Train Epoch 0 [10/74]	Loss: 4.027269
Train Epoch 0 [20/74]	Loss: 3.009481
Train Epoch 0 [30/74]	Loss: 2.154206
Train Epoch 0 [40/74]	Loss: 1.712796
Train Epoch 0 [50/74]	Loss: 1.448546
Train Epoch 0 [60/74]	Loss: 1.374756
Train Epoch 0 [70/74]	Loss: 1.353383
[0] Validation Loss: 1.4281, Accuracy: 62.8378%
Train Epoch 1 [0/74]	Loss: 1.008552
Train Epoch 1 [10/74]	Loss: 0.906048
Train Epoch 1 [20/74]	Loss: 0.510415
Train Epoch 1 [30/74]	Loss: 0.738527
Train Epoch 1 [40/74]	Loss: 0.648519
Train Epoch 1 [50/74]	Loss: 0.449418
Train Epoch 1 [60/74]	Loss: 0.512316
Train Epoch 1 [70/74]	Loss: 0.310407
[1] Validation Loss: 0.9128, Accuracy: 76.3514%
Train Epoch 2 [0/74]	Loss: 0.230892
Train Epoch 2 [10/74]	Loss: 0.127092
Train Epoch 2 [20/74]	Loss: 0.162602
Train Epoch 2 [30/74]	Loss: 0.097363
Train Epoch 2 [40/74]	Loss: 0.185083
Train Epoch 2 [50/74]	Loss: 0.120765
Train Epoch 2 [60/74]	Loss: 0.153913
Train Epoch 2 [70